## B. Jacobiho metoda pro výpočet vlastních čísel a vlastních vektorů symetrické matice

### Napište program pro výpočet vlastních čísel a vlastních vektorů symetrické matice Jacobiho metodou. Programem vypočtěte vlastní čísla a vlastní vektory symetrické matice:

$ [3, -1, 1][-1, 5, -1] [1, -1, 3] $

In [71]:
import numpy as np

Vyhledání Prvku k Rotaci(prvek mimo diagonálu s maximální absolutní hodnotou, který budeme rotovat):

In [72]:
def nejv_nd_prvek(matice):
    pocet_radku = len(matice)
    max_ned_prvek, sum_nediag_prvku, radek, sloupec = 0, 0.0, 0, 0

    for i in range(pocet_radku - 1):
        for j in range(i+1, pocet_radku):
            sum_nediag_prvku = sum_nediag_prvku + matice[i,j]**2 #abs(matice[i,j])  # matice[i,j]**2 ???
            if abs(matice[i,j]) > max_ned_prvek:
                max_ned_prvek = abs(matice[i,j])
                radek = i
                sloupec = j
    return radek, sloupec, sum_nediag_prvku

Výpočet Úhlu Rotace(aby se vybraný prvek po rotaci převedl na nulu):

In [73]:
def vypoc_uhlu(matice, radek, sloupec):
    w = - matice[radek, sloupec]
    m = ( matice[radek,radek] - matice[sloupec, sloupec] ) / 2.0
    v = np.sqrt(w**2.0 + m**2.0)

    c = np.sqrt(( v + abs(m) ) / (2.0 * v))
    s = (np.sign(m) * w) / (2.0 * v * c)
    
    return c, s

Aplikace Rotace: Aplikujeme rotaci na matici I na akumulovanou matici rotací (pro výpočet vlastních vektorů):

In [74]:
def rotace(matice, radek, sloupec, s, c):
    a_ss= matice[sloupec, sloupec]
    a_rr = matice[radek,radek]
    a_rs = matice[radek, sloupec]
    
    for i in range (len(matice)):
        if i!= radek and i!= sloupec:
            matice[i,radek] = matice[radek, i] = c * matice[i, radek] - s * matice[i, sloupec]
            matice[i,sloupec] = matice[sloupec, i] = c * matice[i, sloupec] + s * matice[i, radek]
    
    matice[radek, radek] = c**2 * a_rr + s**2 * a_ss - 2.0 * c * s * a_rs
    matice[sloupec, sloupec] = c**2 * a_ss + s**2 * a_rr + 2.0 *c * s * a_rs
    matice[radek, sloupec] = 0.0
    matice[sloupec,radek] = 0.0
    return matice

In [75]:
def vytv_aktualiz_t(pocet_prvku, radek, sloupec, s, c):
    t = np.eye(pocet_prvku, dtype=np.float64 )
    t[radek,radek] = t[sloupec,sloupec] = c
    t[radek,sloupec]  = s
    t[sloupec, radek] = -s    
    return t

In [76]:
def aktualiz_q(q,t):
    if q is None:
        return t
    else:
        return np.dot(q,t)

celk

In [77]:
def Jakobian(matice, presnost, max_iter):
    
    q = None
    pocet_prvku = len(matice)
    
    for i in range(max_iter):
        
        radek, sloupec, sum_nediag_prvku = nejv_nd_prvek(matice)
        if sum_nediag_prvku*2.0 <= presnost:
            vlastni_cisla = np.diag(matice)
            vlastni_vektory = q
            return vlastni_cisla, vlastni_vektory, i
        
        c,s = vypoc_uhlu(matice, radek, sloupec)
        matice = rotace(matice, radek, sloupec, s, c)
        t = vytv_aktualiz_t(pocet_prvku, radek, sloupec, s, c)
        q = aktualiz_q(q, t)
        
    vlastni_cisla = np.diag(matice)
    vlastni_vektory = q
        
    return vlastni_cisla, vlastni_vektory, i

In [83]:
import math

matice = np.array([
    [ 3, -1,  1], 
    [-1,  5, -1], 
    [ 1, -1,  3]], dtype= 'f' )

presnost = 1e-6
max_iter = 1000

vlastni_cisla, vlastni_vektory, pocet_iteraci = Jakobian(matice, presnost, max_iter)
# VC = [round(i) for i in vlastni_cisla ]
print( f"VC{ vlastni_cisla}" )
print(vlastni_vektory)
print(pocet_iteraci)

VC[2.9927201 5.871053  2.1362262]
[[ 0.54535635 -0.40334227 -0.73477988]
 [ 0.55321532  0.83176819 -0.04598355]
 [ 0.62971364 -0.38141406  0.67674519]]
6


In [80]:
q = None
pocet_prvku = len(matice)

matice = np.array([
    [ 3.0, -1.0,  1.0], 
    [-1.0,  5.0, -1.0], 
    [ 1.0, -1.0,  3.0]])

for i in range(7):
    radek, sloupec, sum_nediag_prvku = nejv_nd_prvek(matice)
    c, s = vypoc_uhlu(matice, radek, sloupec)
    matice = rotace(matice, radek, sloupec, s ,c)
    print(matice)
    t = vytv_aktualiz_t(pocet_prvku, radek, sloupec, s, c)
    q = aktualiz_q(q, t)
    print(i)


[[ 2.58578644  0.          0.5411961 ]
 [ 0.          5.41421356 -1.13098631]
 [ 0.5411961  -1.13098631  3.        ]]
0
[[ 2.58578644 -0.19894343  0.4301723 ]
 [-0.19894343  5.86126422  0.        ]
 [ 0.4301723   0.          2.55294935]]
1
[[ 2.99985341 -0.14333178  0.        ]
 [-0.14333178  5.86126422  0.09939934]
 [ 0.          0.09939934  2.13888238]]
2
[[2.99269166e+00 0.00000000e+00 4.96042288e-03]
 [0.00000000e+00 5.86842597e+00 9.90279484e-02]
 [4.96042288e-03 9.90279484e-02 2.13888238e+00]]
3
[[2.99269166e+00 1.31571585e-04 4.95518782e-03]
 [1.31571585e-04 5.87105354e+00 0.00000000e+00]
 [4.95518782e-03 0.00000000e+00 2.13625481e+00]]
4
[[ 2.99272032e+00  1.31569383e-04  0.00000000e+00]
 [ 1.31569383e-04  5.87105354e+00 -7.61198299e-07]
 [ 0.00000000e+00 -7.61198299e-07  2.13622614e+00]]
5
[[ 2.99272032e+00  0.00000000e+00  3.47945783e-11]
 [ 0.00000000e+00  5.87105354e+00 -7.61198296e-07]
 [ 3.47945783e-11 -7.61198296e-07  2.13622614e+00]]
6
